Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [20]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [23]:
batch_size = 128
num_hidden_neurons = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)


  # Variables.
  hidden_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_neurons]))
  hidden_biases = tf.Variable(tf.zeros([num_hidden_neurons]))
  weights = tf.Variable(
    tf.truncated_normal([num_hidden_neurons, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))

  # Hidden layer computation
  hidden_activation = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases)

  # Training computation.
  logits = tf.matmul(hidden_activation, weights) + biases
  l2_loss = tf.nn.l2_loss(weights) + tf.nn.l2_loss(hidden_weights)
  beta = 0.001
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + beta * l2_loss
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
      tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases), weights) + biases)
  test_prediction = tf.nn.softmax(
      tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases), weights) + biases)

In [8]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 640.167603
Minibatch accuracy: 8.6%
Validation accuracy: 19.0%
Minibatch loss at step 500: 191.215973
Minibatch accuracy: 84.4%
Validation accuracy: 81.3%
Minibatch loss at step 1000: 115.554703
Minibatch accuracy: 82.8%
Validation accuracy: 81.7%
Minibatch loss at step 1500: 69.942444
Minibatch accuracy: 85.9%
Validation accuracy: 83.5%
Minibatch loss at step 2000: 41.364304
Minibatch accuracy: 82.8%
Validation accuracy: 84.9%
Minibatch loss at step 2500: 25.297230
Minibatch accuracy: 85.9%
Validation accuracy: 86.3%
Minibatch loss at step 3000: 15.450619
Minibatch accuracy: 86.7%
Validation accuracy: 87.0%
Test accuracy: 92.8%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [24]:
batch_size = 128
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = ((step % 10) * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 687.089417
Minibatch accuracy: 7.8%
Validation accuracy: 37.1%
Minibatch loss at step 500: 190.168930
Minibatch accuracy: 100.0%
Validation accuracy: 78.5%
Minibatch loss at step 1000: 115.321846
Minibatch accuracy: 100.0%
Validation accuracy: 78.4%
Minibatch loss at step 1500: 69.935730
Minibatch accuracy: 100.0%
Validation accuracy: 78.3%
Minibatch loss at step 2000: 42.413319
Minibatch accuracy: 100.0%
Validation accuracy: 78.3%
Minibatch loss at step 2500: 25.724297
Minibatch accuracy: 100.0%
Validation accuracy: 78.6%
Minibatch loss at step 3000: 15.606077
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Test accuracy: 85.9%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [25]:
batch_size = 128
num_hidden_neurons = 1024

def computation(dataset, weights, biases, is_dropout=False, keep_prob=0.5):
    weight_sum = tf.add(tf.matmul(dataset, weights[0]), biases[0])
    hidden_layer = tf.nn.relu(weight_sum)
    if is_dropout: # ==> add dropout
        hidden_layer = tf.nn.dropout(hidden_layer, keep_prob)
    logits = tf.add(tf.matmul(hidden_layer, weights[1]), biases[1])
    return logits

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  keep_prob = tf.placeholder(tf.float32)

  # Variables.
  weights = [tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_neurons])), 
             tf.Variable(tf.truncated_normal([num_hidden_neurons, num_labels]))]
  biases = [tf.Variable(tf.zeros([num_hidden_neurons])),
            tf.Variable(tf.zeros([num_labels]))]
    
  # Training computation
  logits = computation(tf_train_dataset, weights, biases, is_dropout=True, keep_prob=keep_prob)
  l2_loss = tf.nn.l2_loss(weights[0]) + tf.nn.l2_loss(weights[1])
  beta = 0.001
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + beta * l2_loss
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(computation(tf_valid_dataset, weights, biases))
  test_prediction = tf.nn.softmax(computation(tf_test_dataset, weights, biases))


In [26]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = ((step % 10) * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 855.845642
Minibatch accuracy: 7.0%
Validation accuracy: 27.1%
Minibatch loss at step 500: 192.601089
Minibatch accuracy: 100.0%
Validation accuracy: 80.9%
Minibatch loss at step 1000: 117.774086
Minibatch accuracy: 99.2%
Validation accuracy: 80.6%
Minibatch loss at step 1500: 71.001495
Minibatch accuracy: 99.2%
Validation accuracy: 80.6%
Minibatch loss at step 2000: 43.062965
Minibatch accuracy: 100.0%
Validation accuracy: 81.0%
Minibatch loss at step 2500: 26.120514
Minibatch accuracy: 100.0%
Validation accuracy: 80.8%
Minibatch loss at step 3000: 15.844528
Minibatch accuracy: 100.0%
Validation accuracy: 81.2%
Test accuracy: 87.7%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [32]:
batch_size = 128
hidden_nodes_1 = 1024
hidden_nodes_2 = 500
hidden_nodes_3 = 100
decay_steps = 500
decay_rate = 0.9

def computation(dataset, weights, biases, is_dropout=False, keep_prob=0.7):
    weight_sum_1 = tf.add(tf.matmul(dataset, weights[0]), biases[0])
    hidden_layer_1 = tf.nn.relu(weight_sum_1)
    if is_dropout: # ==> add dropout
        hidden_layer_1 = tf.nn.dropout(hidden_layer_1, keep_prob)
    weight_sum_2 = tf.add(tf.matmul(hidden_layer_1, weights[1]), biases[1])
    hidden_layer_2 = tf.nn.relu(weight_sum_2)
    if is_dropout:
        hidden_layer_2 = tf.nn.dropout(hidden_layer_2, keep_prob)
    weight_sum_3 = tf.add(tf.matmul(hidden_layer_2, weights[2]), biases[2])
    hidden_layer_3 = tf.nn.relu(weight_sum_3)
    if is_dropout:
        hidden_layer_3 = tf.nn.dropout(hidden_layer_3, keep_prob)
    logits = tf.matmul(hidden_layer_3, weights[3]) + biases[3]
    return logits

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  keep_prob = tf.placeholder(tf.float32)

  # Variables.
  weights = [tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes_1], stddev=np.sqrt(2.0 / (image_size * image_size)))), 
             tf.Variable(tf.truncated_normal([hidden_nodes_1, hidden_nodes_2], stddev=np.sqrt(2.0 / hidden_nodes_1))),
             tf.Variable(tf.truncated_normal([hidden_nodes_2, hidden_nodes_3], stddev=np.sqrt(2.0 / hidden_nodes_2))),
             tf.Variable(tf.truncated_normal([hidden_nodes_3, num_labels], stddev=np.sqrt(2.0 / hidden_nodes_3)))]
  biases = [tf.Variable(tf.zeros([hidden_nodes_1])),
            tf.Variable(tf.zeros([hidden_nodes_2])),
            tf.Variable(tf.zeros([hidden_nodes_3])),
            tf.Variable(tf.zeros([num_labels]))]

  global_step = tf.Variable(0) # ==> Add for learning rate decay
    
  # Training computation
  logits = computation(tf_train_dataset, weights, biases, is_dropout=True, keep_prob=keep_prob)
  l2_loss = tf.nn.l2_loss(weights[0]) + tf.nn.l2_loss(weights[1]) + tf.nn.l2_loss(weights[2]) + tf.nn.l2_loss(weights[3])
  beta = 0.001
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + beta * l2_loss
  
  # Optimizer.
  learning_rate = tf.train.exponential_decay(0.5, global_step, decay_steps, decay_rate)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(computation(tf_valid_dataset, weights, biases))
  test_prediction = tf.nn.softmax(computation(tf_test_dataset, weights, biases))

In [34]:
num_steps = 8001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.025173
Minibatch accuracy: 7.8%
Validation accuracy: 19.6%
Minibatch loss at step 500: 1.337509
Minibatch accuracy: 89.1%
Validation accuracy: 83.9%
Minibatch loss at step 1000: 1.104270
Minibatch accuracy: 85.9%
Validation accuracy: 85.1%
Minibatch loss at step 1500: 0.847785
Minibatch accuracy: 91.4%
Validation accuracy: 85.7%
Minibatch loss at step 2000: 1.003792
Minibatch accuracy: 84.4%
Validation accuracy: 86.4%
Minibatch loss at step 2500: 0.881179
Minibatch accuracy: 78.1%
Validation accuracy: 87.0%
Minibatch loss at step 3000: 0.776110
Minibatch accuracy: 84.4%
Validation accuracy: 86.1%
Minibatch loss at step 3500: 0.488276
Minibatch accuracy: 92.2%
Validation accuracy: 87.5%
Minibatch loss at step 4000: 0.717326
Minibatch accuracy: 85.2%
Validation accuracy: 87.9%
Minibatch loss at step 4500: 0.678395
Minibatch accuracy: 86.7%
Validation accuracy: 88.1%
Minibatch loss at step 5000: 0.662016
Minibatch accuracy: 87.5%
Validation accuracy